In [29]:
import os
import torch
from torch import nn
import torchtext
from torchtext.data import get_tokenizer
import numpy as np
import pandas as pd
import json
from torch.utils.data import Dataset , DataLoader

In [30]:
def load_data(path):
    with open(path,'r',encoding='utf-8') as file:
        data = json.load(file)
    # print(len(train_data))
    return data

In [31]:
# import spacy
# nlp = spacy.load('en_core_web_sm')
tokenizer = get_tokenizer('spacy')
train_data =load_data("train.json")
words = set()
for i in train_data:
    sent_arr = set(tokenizer(i['tweet']))
    words = words | sent_arr
words = ["","UNK"]+list(words)
print(len(words))

/home/wujh1123/anaconda3/envs/nlp/lib/python3.8/site-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(


26345


In [32]:
tag_list=["ineffective","unnecessary","pharma","rushed","side-effect","mandatory","country","ingredients","political","none","conspiracy","religious"]
tag_to_idx={}
for i in range(len(tag_list)):
    tag_to_idx[tag_list[i]]=i

In [33]:
print(type(train_data[0]['labels']))
print(train_data[0]['labels'])
print(list(train_data[0]['labels'].keys()))
for i in list(train_data[77]['labels'].keys()):
    print(tag_to_idx[i],i)

<class 'dict'>
{'ineffective': [{'index': 0, 'start': 6, 'end': 10, 'terms': 'cant control the Flu'}, {'index': 1, 'start': 37, 'end': 39, 'terms': 'infectious diseases'}]}
['ineffective']
4 side-effect


In [34]:
vocab2idx = {k:v for v,k in enumerate(words)}
vocab_size = len(words)

In [35]:
def load_glove_vectors(glove_file="glove.6B.300d.txt"):
    word_vectors = {}
    with open(glove_file) as f:
        for line in f :
            s = line.split()
            word_vectors[s[0]] = np.array([float(x) for x in s[1:]])
    return word_vectors

In [36]:
def get_emb_matrix(pretained,words,emb_size=300):
    W = np.zeros((len(words),emb_size),dtype="float32")
    W[0] = np.zeros(emb_size,dtype="float32")
    W[1] = np.random.uniform(-0.25,0.25,emb_size)
    i = 2
    for word in words[2:]:
        if word in pretained:
            W[i] = pretained[word]
        else:
            W[i] =  np.random.uniform(-0.25,0.25,emb_size)
        i+=1
    return W

In [37]:
word_vecs = load_glove_vectors()
pretrain_weights = get_emb_matrix(word_vecs,words)

In [38]:


class CustomDataset(Dataset):
    def __init__(self,data, tokenizer,max_length,words):
        self.data = data 
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.words = words

    def __len__(self):
        return len(self.data)
    def __getitem__(self, index) :
        tweet = self.data[index]["tweet"]
        labels = self.data[index]["labels"]
        encode_text = self.encode(tweet,self.words)
        encode_label = self.label_encoded(labels)
        seq_len = np.count_nonzero(encode_text)
        return encode_text , encode_label , seq_len
    def encode(self,text,vocab2idx):
        token = self.tokenizer(text)
        encoded = np.zeros(self.max_length,dtype=int)
        enc1 = np.array([vocab2idx.get(word,vocab2idx["UNK"]) for word in token] )
        # print(enc1.shape)
        length = min(self.max_length,len(enc1))
        encoded[:length] = enc1[:length]
        return encoded 
    def label_encoded(self,labels):
        encode_label = np.zeros(12,dtype=int)
        list_label = list(labels.keys())
        for i in list_label:
            idx = tag_to_idx[i]
            encode_label[idx] = 1
        # encode_label = torch.LongTensor(encode_label)
        return encode_label 

In [39]:
train_data = CustomDataset(train_data,tokenizer,300,vocab2idx)
train_loader = DataLoader(train_data,batch_size=32)


In [40]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Now is using {device} device")

Now is using cuda device


In [41]:
from torch.nn.utils.rnn import pack_padded_sequence

In [42]:
def train(dataloader,model,loss_fn,optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch , (X,y,seq) in enumerate(dataloader):
        X,y =X.to(device),y.to(device,dtype=torch.float)
        # print(seq)
        pred = model(X,seq)
        # print(pred.dtype)
        loss = loss_fn(pred,y)


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch%200 ==0:
            loss,curr = loss.item() , batch * len(X)
            print(f"loss: {loss:>7f} [{curr:>5d}/{size:>5d}]")

In [43]:
class pretrain_weight_LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim,hidden_dim,pretrain_weights,bidirectional) :
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size,embedding_dim,padding_idx=0)
        self.embeddings.weight.data.copy_(torch.from_numpy(pretrain_weights))
        self.embeddings.weight.requires_grad = True 
        self.lstm = nn.LSTM(embedding_dim,hidden_dim,bidirectional=bidirectional,dropout=0.1,batch_first=True)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(in_features=hidden_dim,out_features=hidden_dim)
        self.droupt = nn.Dropout(0.3)
        self.linear = nn.Linear(hidden_dim,12)
    def forward(self,x,s):
        embeed = self.embeddings(x)
        x_pack = pack_padded_sequence(embeed,s.to('cpu'),batch_first=True, enforce_sorted=False)
        pack_out , (ht,ct) = self.lstm(x_pack)
        out = self.droupt(ht[-1])
        out = self.fc1(out)
        out = self.relu(out)
        out = self.droupt(out)
        out = self.linear(out)
        return out 

In [44]:
bidirectional = False
model_2 = pretrain_weight_LSTM(vocab_size,300,128,pretrain_weights,bidirectional).to(device)

/home/wujh1123/anaconda3/envs/nlp/lib/python3.8/site-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [45]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_2.parameters(),lr=1e-3)

In [46]:
from sklearn.metrics import f1_score

In [47]:
def evalute(dataloader,model,loss_fn):
    model.eval()
    y_pred = []
    y_target = []
    with torch.no_grad():
        for X,y,seq in dataloader:
            X,y = X.to(device) , y.to(device,dtype=torch.float)
            pred = model(X,seq)
            y_pred.extend(torch.sigmoid(pred).cpu().detach().numpy().tolist())
            y_target.extend(y.cpu().detach().numpy().tolist())
    y_preds = (np.array(y_pred)>0.5).astype(int)
    marco_f1= f1_score(y_target,y_preds,average='macro')
    print("marco f1 score : ",marco_f1)
    return marco_f1

In [48]:
val_data =load_data('val.json')
val_data = CustomDataset(val_data,tokenizer,300,vocab2idx)
val_dataloader = DataLoader(val_data,batch_size=1)

# evalute(val_dataloader,model_2,loss_fn)

In [49]:
epochs = 50
max_score = 0 
for t in range(epochs):
    print(f"Epoch {t+1}\n-----------------------")
    train(train_loader,model_2,loss_fn,optimizer)
    score = evalute(val_dataloader,model_2,loss_fn)
    if score > max_score:
        max_score =score
        path = './w_weight.pth'
        torch.save(model_2.state_dict(),path)
        print('sava model')
print("best score: " ,max_score)

Epoch 1
-----------------------
loss: 0.688382 [    0/ 6956]


loss: 0.256707 [ 6400/ 6956]
marco f1 score :  0.0
Epoch 2
-----------------------
loss: 0.306245 [    0/ 6956]
loss: 0.241082 [ 6400/ 6956]
marco f1 score :  0.06512141280353201
sava model
Epoch 3
-----------------------
loss: 0.272564 [    0/ 6956]
loss: 0.201039 [ 6400/ 6956]
marco f1 score :  0.0673903211216644
sava model
Epoch 4
-----------------------
loss: 0.233727 [    0/ 6956]
loss: 0.204813 [ 6400/ 6956]
marco f1 score :  0.10634818027012387
sava model
Epoch 5
-----------------------
loss: 0.209964 [    0/ 6956]
loss: 0.181591 [ 6400/ 6956]
marco f1 score :  0.12417225599618427
sava model
Epoch 6
-----------------------
loss: 0.197581 [    0/ 6956]
loss: 0.159245 [ 6400/ 6956]
marco f1 score :  0.24639098603612533
sava model
Epoch 7
-----------------------
loss: 0.160883 [    0/ 6956]
loss: 0.118408 [ 6400/ 6956]
marco f1 score :  0.29356522872592977
sava model
Epoch 8
-----------------------
loss: 0.117679 [    0/ 6956]
loss: 0.086306 [ 6400/ 6956]
marco f1 score :  0.328217

In [50]:
class testDataset(Dataset):
    def __init__(self,data, tokenizer,max_length,words):
        self.data = data 
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.words = words

    def __len__(self):
        return len(self.data)
    def __getitem__(self, index) :
        tweet = self.data[index]["tweet"]
        encode_text = self.encode(tweet,self.words)
        seq_len = np.count_nonzero(encode_text)
        return encode_text  , seq_len
    def encode(self,text,vocab2idx):
        token = self.tokenizer(text)
        encoded = np.zeros(self.max_length,dtype=int)
        enc1 = np.array([vocab2idx.get(word,vocab2idx["UNK"]) for word in token] )
        # print(enc1.shape)
        length = min(self.max_length,len(enc1))
        encoded[:length] = enc1[:length]
        return encoded 

In [51]:
test_data = load_data("test.json")
test_data = testDataset(test_data,tokenizer,300,vocab2idx)
test_loader = DataLoader(test_data,batch_size=1)

In [52]:
path = './w_weight.pth'
model_2 = pretrain_weight_LSTM(vocab_size,300,128,pretrain_weights,bidirectional).to(device)
model_2.load_state_dict(torch.load(path))
model_2.eval()
y_pred = []
with torch.no_grad():
    for x,seq_len in test_loader:
        x = x.to(device)
        pred = model_2(x,seq_len)
        y_pred.extend(torch.sigmoid(pred).cpu().detach().numpy().tolist())

/home/wujh1123/anaconda3/envs/nlp/lib/python3.8/site-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [53]:
y_preds = (np.array(y_pred) > 0.5).astype(int)
y_preds

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [54]:
df = pd.DataFrame(y_preds, columns=tag_list)
df_reset = df.reset_index()
print(df_reset.head())

   index  ineffective  unnecessary  pharma  rushed  side-effect  mandatory  \
0      0            0            0       1       0            0          0   
1      1            0            0       0       0            0          0   
2      2            0            0       0       0            1          0   
3      3            1            0       0       0            0          0   
4      4            0            0       0       0            0          0   

   country  ingredients  political  none  conspiracy  religious  
0        0            0          0     0           0          0  
1        0            0          0     1           0          0  
2        0            0          0     0           0          0  
3        0            0          0     0           0          0  
4        0            0          0     0           0          0  


In [55]:
data_rows = df_reset.to_dict(orient='records')

In [56]:
import csv
with open("submission.csv", 'w', newline='') as csvfile:
    fieldnames=["index","ineffective","unnecessary","pharma","rushed","side-effect","mandatory","country","ingredients","political","none","conspiracy","religious"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(data_rows)